In [1]:
import numpy as np
import pandas as pd
from scipy import stats, integrate
import matplotlib.pyplot as plt
import re
import seaborn as sns
sns.set(color_codes=True)

import tabula

In [2]:
df_pdf = tabula.read_pdf('https://www.ranepa.ru/images/docs/pk/2022/PP-2022-pril-1.pdf', 
    pages= 'all',
    lattice=True,
    pandas_options={"header": [0, 1, 2, 3]},
    relative_area=True)

df = df_pdf[0]
for df_temp in df_pdf[1:]:
    df = df.append(df_temp,ignore_index=True)

In [3]:
df[[0,1]].head(15)

,0,1
0,Код и наименование\rнаправления подготовки/\rс...,Направленность (профиль/специализация) образов...
1,NaN,NaN
2,1,2
3,БАКАЛАВРИАТ,NaN
4,01.03.02\rПрикладная\rматематика и\rинформатика,Образовательная программа:\rПрикладные цифровы...
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN


In [4]:
df_mos = df[ df[1].str.contains('Образовательная|Совокупность', na=False) ].reset_index()
print('дубликаты: {0}' .format( df_mos[[0,1]].duplicated().sum() ))

df_mos = df_mos[[0,1]].copy().drop_duplicates()

дубликаты: 4


In [5]:
df_mos = df_mos.dropna()
df_one = df_mos[df_mos[1].str.contains('Образовательная')]

def program_name(patern, tmp):
    tmp = tmp.replace('\r'," ").split(':',1)[1]
    tmp = tmp.replace(re.findall(patern, tmp )[-1],'')
    " ".join(tmp.split())
    tmp = tmp.strip()
    return tmp

def facult_name(patern, tmp):
    tmp = tmp.replace('\r'," ")
    tmp = re.findall(patern, tmp )[-1].strip()
    return tmp

def std_name(tmp):
    tmp = tmp.replace('\r'," ")
    return tmp

df_one_mos = pd.DataFrame([])
df_one_mos['код'] = df_one.iloc[:, 0].map(lambda x: x.split('\r',1)[0])
df_one_mos['стандарт'] = df_one.iloc[:,0].map(lambda x: x.split('\r',1)[1])
df_one_mos['программа'] =  df_one.iloc[:,1].map(lambda x: program_name("\([ а-яa-zА-ЯA-Z0-9-: ]*\)", x))
df_one_mos['подразделение'] =  df_one.iloc[:,1].map(lambda x: facult_name("\(([ а-яa-zА-ЯA-Z0-9-: ]*)\)", x))
df_one_mos['программа'] = df_one_mos['программа'].map(lambda x: re.sub('[ё]', 'е', x))
df_one_mos['стандарт'] = df_one_mos['стандарт'].map(lambda x: std_name(x))
df_one_mos

,код,стандарт,программа,подразделение
0,01.03.02,Прикладная математика и информатика,Прикладные цифровые технологии,Школа ИТ- менеджмента ЭМИТ
2,09.03.03,Прикладная информатика,Прикладная информатика в цифровой экономике,ФИТАД ЭМИТ
3,09.03.03,Прикладная информатика,Системы искусственного интеллекта (прикладная ...,ФИТАД ЭМИТ
4,09.03.03,Прикладная информатика,Анализ данных,ФИТАД ЭМИТ
5,09.03.03,Прикладная информатика,Веб-разработка,ФИТАД ЭМИТ
...,...,...,...,...
104,43.03.03,Гостиничное дело,Управление гостиничным бизнесом,ИОМ
105,46.03.01,История,История государства и власти (Liberal Arts),ИОН
106,50.03.01,Искусства и гуманитарные науки,Филология и перевод (Liberal Arts),ИОН
107,54.03.01,Дизайн,Современный дизайн,ИОН


In [6]:
df_multi = df_mos[df_mos[1].str.contains('Совокупность')]

def multi_transform(tmp):
    tmp_3 = tmp.iloc[1].split(':\r',1)[1]
    tmp_3 = " ".join(tmp_3.split())
    tmp_3 = re.split('\d.',tmp_3)[1:]
    tmp_3 = np.char.replace(tmp_3,'\r',' ')
    tmp_3 = np.char.replace(tmp_3,'.','')
    dic = {1: tmp.iloc[0].split('\r',1)[0],
           2: tmp.iloc[0].split('\r',1)[1],
           3: tmp_3
          }
    df_tmp = pd.DataFrame(dic)
    df_tmp[2] = df_tmp[2].map(lambda x: std_name(x))
    df_tmp[4] = df_tmp[3].map(lambda x: facult_name("\(([ а-яa-zА-ЯA-Z0-9-: ]*)\)", x))
    df_tmp[3] = df_tmp[3].map(lambda x: x.replace(re.findall("\([ а-яa-zА-ЯA-Z0-9-: ]*\)", x )[-1],'').strip() )
    return df_tmp

tmp = [ multi_transform(df_multi.loc[idx]) for idx in df_multi.index.values]
df_multi_mos = pd.concat(tmp, ignore_index=True)
df_multi_mos.columns = df_one_mos.columns.values
display(df_multi_mos.head(3))


,код,стандарт,программа,подразделение
0,09.03.03,Прикладная информатика,Прикладная информатика в экономике,ФИТАД ЭМИТ
1,09.03.03,Прикладная информатика,Цифровые технологии,ФИТАД ЭМИТ
2,09.03.03,Прикладная информатика,Прикладная информатика в информационной безопа...,ФИТАД ЭМИТ


In [7]:
df_moscow = pd.concat([df_one_mos, df_multi_mos], ignore_index=True)
df_moscow.drop_duplicates(inplace=True)
df_moscow.tail(15)

,код,стандарт,программа,подразделение
147,41.03.01,Зарубежное регионоведение,Управление международными проектами и программ...,ИГСУ
148,41.03.01,Зарубежное регионоведение,Управление международными проектами и программ...,ИГСУ
149,41.03.02,Регионоведение России,Региональная политика и региональное управлени...,ИГСУ
150,41.03.02,Регионоведение России,Развитие международного бизнеса в регионах России,ИГСУ
151,42.03.01,Реклама и связи с общественностью,Диджитал реклама и связи с общественностью,ИОН
152,42.03.01,Реклама и связи с общественностью,Управление социальными коммуникациями (Liberal...,ИОН
153,42.03.01,Реклама и связи с общественностью,Цифровые медиа,ИОМ
154,42.03.01,Реклама и связи с общественностью,Реклама и PR,ИОМ
156,42.03.02,Журналистика,Политическая журналистика,ИГСУ
157,38.05.01,Экономическая безопасность,Экономико-правовое обеспечение экономической б...,ФНБ ИПНБ


In [8]:
df_pdf = tabula.read_pdf('https://www.ranepa.ru/images/docs/pk/2022/PP-2022-pril-2_fil.pdf', 
    pages= 'all',
    lattice=True,
    pandas_options={"header": [0, 1, 2, 3]},
    relative_area=True)

df = df_pdf[0]
for df_temp in df_pdf[1:]:
    df = df.append(df_temp,ignore_index=True)
df.head(10) 
    
searchfor = ['Направление','Общие',"очное"]
mask = df[1].str.contains('|'.join(searchfor), na=False)|df[2].str.contains('|'.join(searchfor), na=False)
df = df[~mask][[0,2]].copy()
#mask_uslovie = df[0].str.contains("УСЛОВИЯ", na=False)


Got stderr: дек 29, 2021 10:33:54 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode



In [9]:
df['f'] = df[df[0].str.contains('ФИЛИАЛ').replace(np.nan, False)][0]
df['f'] = df['f'].ffill()
df_f = df.dropna()
df1 = df_f[df_f[2].str.contains('Образовательная')]
df1.head()

,0,2,f
5,37.03.01\rПсихология,Образовательная программа:\rПсихология управления,АЛТАЙСКИЙ ФИЛИАЛ
8,37.03.01\rПсихология,Образовательная программа:\rПсихология управления,АЛТАЙСКИЙ ФИЛИАЛ
17,38.03.04\rГосударственное и\rмуниципальное упр...,Образовательная программа:\rАдминистративное у...,АЛТАЙСКИЙ ФИЛИАЛ
20,38.03.04\rГосударственное и\rмуниципальное упр...,Образовательная программа:\rАдминистративное у...,АЛТАЙСКИЙ ФИЛИАЛ
30,42.03.05\rМедиакоммуникации,Образовательная программа:\rМедиакоммуникации ...,АЛТАЙСКИЙ ФИЛИАЛ


In [10]:
df1f = pd.DataFrame([])
df1f['код'] = df1.iloc[:, 0].map(lambda x: x.split('\r',1)[0])
df1f['стандарт'] = df1.iloc[:,0].map(lambda x: x.split('\r',1)[1])
df1f['программа'] =  df1.iloc[:,1].map(lambda x: x.split('\r',1)[1])
df1f['подразделение'] =  df1.iloc[:,2]
df1f['программа'] = df1f['программа'].map(lambda x: re.sub('[ё]', 'е', x))
df1f['программа'] = df1f['программа'].map(lambda x: std_name(x))
df1f['стандарт'] = df1f['стандарт'].map(lambda x: std_name(x))
df1f.head()

,код,стандарт,программа,подразделение
5,37.03.01,Психология,Психология управления,АЛТАЙСКИЙ ФИЛИАЛ
8,37.03.01,Психология,Психология управления,АЛТАЙСКИЙ ФИЛИАЛ
17,38.03.04,Государственное и муниципальное управление,Административное управление,АЛТАЙСКИЙ ФИЛИАЛ
20,38.03.04,Государственное и муниципальное управление,Административное управление,АЛТАЙСКИЙ ФИЛИАЛ
30,42.03.05,Медиакоммуникации,Медиакоммуникации и продвижение в цифровой среде,АЛТАЙСКИЙ ФИЛИАЛ


In [11]:
df_multi_f = df_f[df_f[2].str.contains('Совокупность')]
df_multi_f.head()


,0,2,f
11,38.03.01\rЭкономика,Совокупность образовательных программ:\r1. Эко...,АЛТАЙСКИЙ ФИЛИАЛ
14,38.03.01\rЭкономика,Совокупность образовательных программ:\r1. Эко...,АЛТАЙСКИЙ ФИЛИАЛ
23,40.03.01\rЮриспруденция,Совокупность образовательных программ:\r1. Уго...,АЛТАЙСКИЙ ФИЛИАЛ
26,40.03.01\rЮриспруденция,Совокупность образовательных программ:\r1. Уго...,АЛТАЙСКИЙ ФИЛИАЛ
44,38.03.04\rГосударственное и\rмуниципальное упр...,Совокупность образовательных программ:\r1. Адм...,АЛТАЙСКИЙ ФИЛИАЛ


In [12]:
def multi_transform(tmp):
    tmp_3 = tmp.iloc[1].split(':\r',1)[1]
    tmp_3 = " ".join(tmp_3.split())
    tmp_3 = re.split('\d.',tmp_3)[1:]
    tmp_3 = np.char.replace(tmp_3,'\r',' ')
    tmp_3 = np.char.replace(tmp_3,'.','')
    dic = {1: tmp.iloc[0].split('\r',1)[0],
           2: tmp.iloc[0].split('\r',1)[1],
           3: tmp_3,
           4: tmp.iloc[2]
          }
    df_tmp = pd.DataFrame(dic)
    df_tmp[2] = df_tmp[2].map(lambda x: std_name(x))
    #df_tmp[3] = df_tmp[3].map(lambda x: x.replace(re.findall("\([ а-яa-zА-ЯA-Z0-9-: ]*\)", x )[-1],'').strip() )
    return df_tmp

tmp = [ multi_transform(df_multi_f.loc[idx]) for idx in df_multi_f.index.values]
df_multi_fili = pd.concat(tmp, ignore_index=True)
df_multi_fili.columns = df_one_mos.columns.values
#display(df_multi_fili.tail(10))

In [13]:
df_fili = pd.concat([df1f, df_multi_fili], ignore_index=True)
df_fili.drop_duplicates(inplace=True)
df_fili.head()

,код,стандарт,программа,подразделение
0,37.03.01,Психология,Психология управления,АЛТАЙСКИЙ ФИЛИАЛ
2,38.03.04,Государственное и муниципальное управление,Административное управление,АЛТАЙСКИЙ ФИЛИАЛ
4,42.03.05,Медиакоммуникации,Медиакоммуникации и продвижение в цифровой среде,АЛТАЙСКИЙ ФИЛИАЛ
6,38.03.01,Экономика,Экономика и управление финансами,АЛТАЙСКИЙ ФИЛИАЛ
8,40.05.04,Судебная и прокурорская деятельность,Прокурорская деятельность,АЛТАЙСКИЙ ФИЛИАЛ


# Перечень всех программ Академии

In [14]:
df_full = pd.concat([df_moscow, df_fili])
with pd.option_context('display.max_rows', None):
         display(df_full)

,код,стандарт,программа,подразделение
0,01.03.02,Прикладная математика и информатика,Прикладные цифровые технологии,Школа ИТ- менеджмента ЭМИТ
1,09.03.03,Прикладная информатика,Прикладная информатика в цифровой экономике,ФИТАД ЭМИТ
2,09.03.03,Прикладная информатика,Системы искусственного интеллекта (прикладная ...,ФИТАД ЭМИТ
3,09.03.03,Прикладная информатика,Анализ данных,ФИТАД ЭМИТ
4,09.03.03,Прикладная информатика,Веб-разработка,ФИТАД ЭМИТ
5,09.03.03,Прикладная информатика,Прикладная информатика в экономике,ФИТАД ЭМИТ
6,09.03.03,Прикладная информатика,Прикладная информатика в энергетических системах,ФИТАД ЭМИТ
7,27.03.05,Инноватика,Технологическое предпринимательство,ИОМ
8,37.03.01,Психология,Психология: современная теория и практика (Lib...,ИОН
9,37.03.01,Психология,Психологическое консультирование и коучинг,ИОН


In [20]:
#Программы по экономике
tmp = df_full[df_full['код']=='38.03.01'].sort_values(by='программа')
with pd.option_context('display.max_colwidth', None):
         display(tmp)

,код,стандарт,программа,подразделение
532,38.03.01,Экономика,Бухгалтерский учет,БАЛАКОВСКИЙ ФИЛИАЛ
536,38.03.01,Экономика**,Бухгалтерский учет,БАЛАКОВСКИЙ ФИЛИАЛ
546,38.03.01,Экономика,"Бухгалтерский учет, анализ и аудит",ДЗЕРЖИНСКИЙ ФИЛИАЛ
548,38.03.01,Экономика**,"Бухгалтерский учет, анализ и аудит",ДЗЕРЖИНСКИЙ ФИЛИАЛ
627,38.03.01,Экономика,Государственные и муниципальные финансы,СИБИРСКИЙ ИНСТИТУТ УПРАВЛЕНИЯ - ФИЛИАЛ
...,...,...,...,...
461,38.03.01,Экономика,Экономика предприятий и организаций,УРАЛЬСКИЙ ИНСТИТУТ УПРАВЛЕНИЯ - ФИЛИАЛ
44,38.03.01,Экономика,Экономика предприятий и организаций,ВЛАДИМИРСКИЙ ФИЛИАЛ
91,38.03.01,Экономика,Экономика предприятий и организаций,ЭФ ЭМИТ
459,38.03.01,Экономика,Экономика умного города,УРАЛЬСКИЙ ИНСТИТУТ УПРАВЛЕНИЯ - ФИЛИАЛ


In [15]:
#df_full.to_excel("bach_2022.xlsx")